In [67]:
from pymongo import MongoClient
import pygsheets
import os
from tqdm import tqdm
import pandas as pd

In [21]:
gs_client = pygsheets.authorize(service_file=os.environ["GS_CRED_PATH"])

In [22]:
sh = gs_client.open('moving_data')
worksheet = sh.sheet1 

In [24]:
df = worksheet.get_as_df()

In [25]:
df.head()

,url,author,comment_id,upvotes,spotify_id,song_match,artist_match,post_id
0,NaN,thunderstorm-nigg,fi72l72,326,561F1zqRwGPCTMRsLsXVtL,Time in bottle,Jim Croce,f6qlck
1,NaN,hockeyxfight,fi78pa1,2976,6mFkJmJqdDVQ1REhVfGgd1,Wish You Were Here,Pink Floyd,f6qlck
2,NaN,Zyffrin,fi6laxj,8598,0QZ5yyl6B6utIWkxeBDxQN,The Night We Met,Lord Huron,f6qlck
3,NaN,Talstone,fi6hjh2,2043,0MNNKSUU9OOQ8DSGWduw79,Mystery of Love,Sufjan Stevens,f6qlck
4,NaN,_THATSALOTADAMAGE_,fi78ygl,1201,0GgN4MhR5GKn5IcKN0e0rG,Cancer,My Chemical Romance,f6qlck


In [36]:
client = MongoClient('localhost', 27017)

In [37]:
db = client.mmt_project

In [46]:
post_ids = df["post_id"]
comment_ids = df["comment_id"]

In [65]:
comment_texts = []
for i in tqdm(range(len(comment_ids))):
    document = db[post_ids[i]].find_one({"comment_id": comment_ids[i]})
    body = document["body"]
    comment_texts.append(body)

100%|██████████████████████████████████████████████████████████████████| 8552/8552 [00:37<00:00, 230.34it/s]


In [70]:
df_comments = pd.DataFrame({
    "comment_id": comment_ids,
    "body": comment_texts
})

In [71]:
df_comments.to_json("comments_text.json")

In [72]:
pd.read_json("comments_text.json")

,comment_id,body
0,fi72l72,Time in bottle by Jim Croce
1,fi78pa1,Wish You Were Here by Pink Floyd. I don’t real...
2,fi6laxj,The Night We Met - Lord Huron
3,fi6hjh2,Mystery of Love by Sufjan Stevens. I can’t eve...
4,fi78ygl,"Cancer by My Chemical Romance, my Aunt died of..."
...,...,...
8547,co4xr00,"in case you haven't heard it yet, here's Danny..."
8548,co4xr00,"in case you haven't heard it yet, here's Danny..."
8549,co5bxsn,[Deftones](https://www.youtube.com/watch?v=59c...
8550,co5ekkd,[This](https://www.youtube.com/watch?v=vmd1qMN...
